# Is Model-X Gaussian knockoffs the same as Matteo's knockoffs?

In [1]:
using Revise
using Knockoffs
using Test
using LinearAlgebra
using Random
using StatsBase
using Statistics
using Distributions
using ToeplitzMatrices
using RCall
using PositiveFactorizations
using UnicodePlots
using MATLAB
using SCS
using JuMP
using Convex

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1423


Our model is

$$X_{p \times 1} \sim N(\mathbf{0}_p, \Sigma)$$
where
$$
\Sigma = 
\begin{pmatrix}
    1 & \rho & \rho^2 & ... & \rho^p\\
    \rho & \rho^2 & & ... & \rho^{p-1}\\
    \vdots & & & \rho^2 & \vdots \\
    \rho^p & \cdots & & & 1
\end{pmatrix}
$$
Given $n$ iid samples from the above distribution, we will generate knockoffs according to 
$$(X, \tilde{X}) \sim N
\left(0, \ 
\begin{pmatrix}
    \Sigma & \Sigma - diag(s)\\
    \Sigma - diag(s) & \Sigma
\end{pmatrix}
\right)
$$
where $s$ is solved so that $0 \le s_j \le 1 \forall j$ and $G$ is PSD (i.e. $2Σ - diag(s)$ is PSD)

In [13]:
# simulate data
Random.seed!(2022)
n = 100
p = 200
ρ = 0.4
Sigma = Matrix(SymmetricToeplitz(ρ.^(0:(p-1))))
L = cholesky(Sigma).L
X = randn(n, p) * L # var(X) = L var(N(0, 1)) L' = var(Σ)

# generate Julia knockoff
true_mu = zeros(p)
@time knockoff = modelX_gaussian_knockoffs(X, :sdp, true_mu, Sigma) # precompile
X = knockoff.X
X̃ = knockoff.X̃
s = knockoff.s;

  2.925068 seconds (587.09 k allocations: 405.708 MiB, 1.59% gc time)


In [14]:
# compare with Matteo's knockoffs in R
@rput Sigma X true_mu
@time begin
    R"""
    library(knockoff)
    r_s = create.solve_sdp(Sigma)
    X_ko = create.gaussian(X, true_mu, Sigma, method = "sdp", diag_s=r_s)
    """
end
@rget r_s X_ko

  1.356420 seconds (54 allocations: 1.328 KiB)


100×200 Matrix{Float64}:
 -0.896207   -1.40432    -0.641874  …   0.590455   0.239027  -0.441005
  0.808881    1.25478     1.36092      -1.27792   -1.26546   -1.41308
 -1.76738    -0.676855   -0.489366     -1.78298   -0.518164  -0.469071
 -0.0207238  -0.0987778  -0.612011      0.867469   0.961678   0.741441
 -1.30192    -2.29895    -0.626801      1.13025   -0.55899   -0.589619
 -0.795424   -1.3748      0.924679  …   0.140832   0.300213   0.898909
 -0.499375    0.105092   -1.47103      -0.832738   0.682917  -0.0156104
  0.152931   -0.399363   -0.288129      0.59081    0.889807  -0.497541
 -1.23746    -0.891151   -0.15415      -0.358655  -0.447035  -1.19964
  1.00867     1.36912     1.16893       0.353287  -2.36672   -1.57729
 -0.359748   -0.435782   -0.771414  …   0.185919   0.169166   0.267265
  0.919273    0.644287    0.77185      -0.698598  -0.562157  -0.403914
 -1.38459    -2.54757    -0.771107     -1.2857    -0.716831   0.124052
  ⋮                                 ⋱                 

In [7]:
# compare with Matteo's knockoffs in MATLAB
@mput Sigma X true_mu
@time begin
    mat"""
    addpath("/Users/biona001/Documents/MATLAB/knockoffs_matlab")
    matlab_s = knockoffs.create.solveSDP(Sigma)
    """
end
@mget matlab_s

  0.929914 seconds (7 allocations: 192 bytes)


200-element Vector{Float64}:
 0.9999999896707443
 0.9714281030691082
 0.8114293804808618
 0.8754281642668925
 0.8498287272443871
 0.8600684790356232
 0.8559725790293014
 0.8576109226326499
 0.8569555851247777
 0.8572177205479895
 0.8571128718268968
 0.8571548177601286
 0.857138046821755
 ⋮
 0.8571548177626378
 0.8571128718285126
 0.857217720541555
 0.8569555851312949
 0.8576109226274933
 0.8559725790350241
 0.8600684790299399
 0.8498287272488133
 0.875428164264091
 0.8114293804802359
 0.971428103070526
 0.9999999896707444

Compare values of $s$

In [15]:
# s = from Knockoffs.jl, r_s from knockoff R package, matlab_s from knockoff MATLAB package
[s r_s matlab_s]

200×3 Matrix{Float64}:
 1.0       1.0       1.0
 0.971429  0.971427  0.971428
 0.811429  0.811431  0.811429
 0.875429  0.875427  0.875428
 0.849829  0.849829  0.849829
 0.860069  0.860068  0.860068
 0.855973  0.855973  0.855973
 0.857611  0.857611  0.857611
 0.856956  0.856956  0.856956
 0.857218  0.857218  0.857218
 0.857113  0.857113  0.857113
 0.857155  0.857155  0.857155
 0.857138  0.857138  0.857138
 ⋮                   
 0.857155  0.857155  0.857155
 0.857113  0.857113  0.857113
 0.857218  0.857218  0.857218
 0.856956  0.856956  0.856956
 0.857611  0.857611  0.857611
 0.855973  0.855973  0.855973
 0.860069  0.860068  0.860068
 0.849829  0.849829  0.849829
 0.875429  0.875427  0.875428
 0.811429  0.811431  0.811429
 0.971429  0.971427  0.971428
 1.0       1.0       1.0

Compare knockoffs

In [41]:
r2 = [cor(X[:, i], X_ko[:, i]) for i in 1:p]
histogram(r2)

┌ Info: Precompiling UnicodePlots [b8865327-cd53-5732-bb35-84acbb429228]
└ @ Base loading.jl:1423


                ┌                                        ┐ 
   [-0.2, -0.1) ┤▌ 1                                       
   [-0.1,  0.0) ┤████████▎ 16                              
   [ 0.0,  0.1) ┤█████████████████████████████▍ 57         
   [ 0.1,  0.2) ┤████████████████████████████████████  70  
   [ 0.2,  0.3) ┤██████████████████████████▎ 51            
   [ 0.3,  0.4) ┤██▏ 4                                     
   [ 0.4,  0.5) ┤▌ 1                                       
                └                                        ┘ 
                                 Frequency                 

In [42]:
r2 = [cor(X[:, i], X̃[:, i]) for i in 1:p]
histogram(r2)

                ┌                                        ┐ 
   [-0.2, -0.1) ┤▌ 1                                       
   [-0.1,  0.0) ┤████▍ 9                                   
   [ 0.0,  0.1) ┤██████████████████████▌ 47                
   [ 0.1,  0.2) ┤████████████████████████████████████  75  
   [ 0.2,  0.3) ┤███████████████████████████▍ 57           
   [ 0.3,  0.4) ┤████▊ 10                                  
   [ 0.4,  0.5) ┤▌ 1                                       
                └                                        ┘ 
                                 Frequency                 

In [3]:
using LinearAlgebra
using Random
using ToeplitzMatrices
using MATLAB
using Convex
using SCS

# simulate data
Random.seed!(2022)
n = 100
p = 200
ρ = 0.4
Sigma = Matrix(SymmetricToeplitz(ρ.^(0:(p-1))))
L = cholesky(Sigma).L
X = randn(n, p) * L;

In [10]:
# solve SDP using Convex.jl
function julia_sdp(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    solve!(problem, SCS.Optimizer())
    return evaluate(svar)
end
@time s_julia = julia_sdp(Sigma)

 97.910649 seconds (882.54 k allocations: 87.790 MiB, 0.01% gc time, 0.04% compilation time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 40201, constraints m: 80401
cones: 	  z: primal zero / dual free vars: 59901
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 100701, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 8.69e+00  9.98e-01  1.69e+03 -1.09e+03

200-element Vector{Float64}:
 0.9999831968499809
 0.9713101250950568
 0.8111610020741279
 0.8753851680693364
 0.8499617029261299
 0.8604197007608251
 0.8565600229194104
 0.8584213370586803
 0.8579840302238552
 0.8584434210925962
 0.8585231428041151
 0.8587359710406687
 0.858880681212524
 ⋮
 0.8587359710406745
 0.8585231428041343
 0.8584434210926003
 0.8579840302238578
 0.8584213370586931
 0.8565600229194121
 0.8604197007608134
 0.8499617029261346
 0.875385168069348
 0.8111610020741296
 0.9713101250950765
 0.9999831968499767

In [7]:
# solve SDP using Matlab's CVX
function matlab_sdp(Sigma::Matrix)
    p = Float64(size(Sigma, 1))
    @mput Sigma p
    mat"""
    cvx_begin
        variable s(p);
        maximize(sum(s));
        2*Sigma-diag(s) == semidefinite(p);
        0 <= s <= 1;
    cvx_end
    """
    @mget s
    return s
end
@time s_matlab = matlab_sdp(Sigma)

>> >> >> >> >> >> >> >>  
Calling SDPT3 4.0: 20500 variables, 200 equality constraints
   For improved efficiency, SDPT3 is solving the dual problem.
------------------------------------------------------------

 num. of constraints = 200
 dim. of sdp    var  = 200,   num. of sdp  blk  =  1
 dim. of linear var  = 400
*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|1.9e+02|1.6e+01|1.5e+06| 8.400000e+04  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|0.524|4.4e-05|7.8e+00|6.2e+05| 4.042192e+04 -1.699825e+03| 0:0:00| chol  1  1 
 2|1.000|1.000|5.2e-05|5.9e-02|3.7e+04| 3.332107e+04 -6.961080e-01| 0:0:00| chol  1  1 
 3|0.9

200-element Vector{Float64}:
 0.9999999996707444
 0.9714281127833894
 0.8114293885951557
 0.8754281730211743
 0.8498287357426745
 0.8600684876363082
 0.8559725875890273
 0.8576109312087593
 0.8569555936943336
 0.8572177291201668
 0.8571128803980256
 0.8571548263316769
 0.8571380553931356
 ⋮
 0.8571548263341862
 0.8571128803996414
 0.8572177291137323
 0.8569555937008508
 0.8576109312036027
 0.85597258759475
 0.8600684876306248
 0.8498287357471007
 0.8754281730183727
 0.8114293885945298
 0.9714281127848071
 0.9999999996707445

In [8]:
[s_julia s_matlab]

200×2 Matrix{Float64}:
 0.999983  1.0
 0.97131   0.971428
 0.811161  0.811429
 0.875385  0.875428
 0.849962  0.849829
 0.86042   0.860068
 0.85656   0.855973
 0.858421  0.857611
 0.857984  0.856956
 0.858443  0.857218
 0.858523  0.857113
 0.858736  0.857155
 0.858881  0.857138
 ⋮         
 0.858736  0.857155
 0.858523  0.857113
 0.858443  0.857218
 0.857984  0.856956
 0.858421  0.857611
 0.85656   0.855973
 0.86042   0.860068
 0.849962  0.849829
 0.875385  0.875428
 0.811161  0.811429
 0.97131   0.971428
 0.999983  1.0

In [22]:
# solve SDP using Convex.jl
function julia_sdp(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    optim = SCS.Optimizer()
    Convex.MOI.set(optim, Convex.MOI.RawParameter("max_iters"), 100)
    solve!(problem, optim)
    return evaluate(svar)
end
@time s_julia = julia_sdp(Sigma)

  1.922261 seconds (887.02 k allocations: 88.041 MiB, 0.93% gc time, 1.58% compilation time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 40201, constraints m: 80401
cones: 	  z: primal zero / dual free vars: 59901
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 100701, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 8.69e+00  9.98e-01  1.69e+03 -1.09e+03  1

┌ Warning: RawParameter(::String is deprecated. Use `RawOptimizerAttribute(::String)` instead.
└ @ MathOptInterface /Users/biona001/.julia/packages/MathOptInterface/IIN1o/src/deprecate.jl:47
┌ Warning: Problem status ALMOST_OPTIMAL; solution may be inaccurate.
└ @ Convex /Users/biona001/.julia/packages/Convex/uI27T/src/solution.jl:263


200-element Vector{Float64}:
 0.9690156119283567
 0.9660616900738638
 0.9569083622663112
 0.9542460233526939
 0.958683954537698
 0.9586313639069278
 0.9558922980615159
 0.957296212052635
 0.958494384920848
 0.9567913873057917
 0.9568463043236631
 0.958161102149923
 0.957327548941899
 ⋮
 0.9581611021499161
 0.9568463043236636
 0.9567913873057902
 0.9584943849208619
 0.9572962120526427
 0.9558922980615295
 0.9586313639069406
 0.9586839545376967
 0.9542460233526928
 0.9569083622663029
 0.9660616900738732
 0.9690156119283562

## JuMP + SCS

In [5]:
# Build model via JuMP
model = Model(SCS.Optimizer)
@variable(model, 0 ≤ s[i = 1:p] ≤ 1)
@objective(model, Max, sum(s))
@constraint(model, Symmetric(2*Sigma - diagm(s[1:p])) in PSDCone());

# Solve optimization problem
@time JuMP.optimize!(model)

# Retrieve solution
JuMP.value.(s)

176.093887 seconds (13.94 M allocations: 824.327 MiB, 0.12% gc time, 4.00% compilation time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 200, constraints m: 20500
cones: 	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 600, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.57e+00  2.60e+00  1.52e+02 -8.39e+02  1.00e-01  2.06e-02 
   250| 1.31e-03  4.49e-0

200-element Vector{Float64}:
 0.9999763341839858
 0.9714721431613923
 0.811437900544127
 0.8754554276955425
 0.8498581040680208
 0.8601108747149225
 0.8560269732919821
 0.8576794828209612
 0.8570387972293917
 0.8573157730959631
 0.8572252358503829
 0.8572804382118645
 0.8572756122633942
 ⋮
 0.8572804382118562
 0.8572252358503814
 0.8573157730959605
 0.8570387972293837
 0.857679482820951
 0.8560269732919861
 0.8601108747149093
 0.8498581040680084
 0.8754554276955282
 0.8114379005441317
 0.9714721431613778
 0.9999763341839905

## Hypatia solver

In [14]:
using Hypatia

function julia_sdp(Σ::Matrix)
    s = Variable(size(Σ,1), Convex.Positive())
    add_constraint!(s, s ≤ 1)
    constraint = 2*Σ - diagm(s) ⪰ 0
    problem = maximize(sum(s),constraint)
    #solve!(problem, SCS.Optimizer; silent_solver = true)
    #solve!(problem, () -> Dualization.dual_optimizer(SCS.Optimizer))
    solve!(problem, Hypatia.Optimizer; silent_solver = true)
    return evaluate(s)  
end

@time s_julia = julia_sdp(Sigma) # compile
@time s_julia = julia_sdp(Sigma) 

  3.402519 seconds (1.57 M allocations: 470.478 MiB, 9.29% gc time, 16.16% compilation time)
  3.183560 seconds (587.13 k allocations: 403.170 MiB, 4.28% gc time)


200-element Vector{Float64}:
 0.9999999975421989
 0.9714285562750639
 0.8114286000388236
 0.8754285608972914
 0.8498285802412486
 0.8600685709048966
 0.8559725752335535
 0.8576109732729927
 0.856955614169666
 0.8572177577255956
 0.8571129003675105
 0.8571548432807583
 0.8571380661077549
 ⋮
 0.8571548433098494
 0.8571129003234808
 0.8572177577571737
 0.8569556141700471
 0.8576109732517373
 0.8559725752488924
 0.860068570920087
 0.8498285802136867
 0.8754285609148045
 0.8114286000369729
 0.9714285562700409
 0.9999999975421956

## Convex + SCS + Dualization

In [19]:
using Dualization

# solve SDP using Convex.jl
function julia_dual(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    solve!(problem, Dualization.dual_optimizer(SCS.Optimizer))
#     model = Model(dual_optimizer(optimizer_with_attributes(SCS.Optimizer, "maxit" => 5)))
#     solve!(problem, model)
    return evaluate(svar)
end
@time s_dual = julia_dual(Sigma) # compile
@time s_dual = julia_dual(Sigma)

  9.215603 seconds (2.61 M allocations: 222.878 MiB, 0.49% gc time, 0.32% compilation time)
 11.798738 seconds (2.53 M allocations: 218.782 MiB, 0.38% gc time)
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 80401, constraints m: 60701
cones: 	  z: primal zero / dual free vars: 40201
	  l: linear vars: 400
	  s: psd vars: 20100, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct
	  nnz(A): 121201, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
----------------------------------------------

200-element Vector{Float64}:
 0.9999831629523064
 0.9717954065607048
 0.8114913130609391
 0.8756237620958564
 0.8500218563405836
 0.8602819306245469
 0.856259951902612
 0.8580445905725752
 0.8575234851306435
 0.8577796795551333
 0.8575871497479202
 0.8576717044215091
 0.8579081820055083
 ⋮
 0.8576717044215467
 0.8575871497479558
 0.8577796795551594
 0.8575234851306626
 0.8580445905725903
 0.8562599519026469
 0.8602819306245854
 0.8500218563406028
 0.875623762095871
 0.8114913130609658
 0.971795406560729
 0.9999831629523045

## JuMP + SCS + Dualization

In [ ]:
using Dualization

# Build model via JuMP
model = Model(dual_optimizer(SCS.Optimizer))
@variable(model, 0 ≤ s[i = 1:p] ≤ 1)
@objective(model, Max, sum(s))
@constraint(model, 2*Sigma - Diagonal(s) in PSDCone());

# Convert to dual model and solve
# dual_model = dualize(model)
@time JuMP.optimize!(model)

# Retrieve solution
JuMP.value.(s)

## Convex + Hypatia + dualization

In [ ]:
# solve SDP using Convex.jl
function julia_dual(Sigma::Matrix)
    p = size(Sigma, 1)
    svar = Variable(p)
    problem = maximize(sum(svar), svar ≥ 0, 1 ≥ svar, 2*Sigma - Diagonal(svar) == Semidefinite(p))
    solve!(problem, Dualization.dual_optimizer(Hypatia.Optimizer))
    return evaluate(svar)
end
@time s_dual = julia_dual(Sigma) # compile
@time s_dual = julia_dual(Sigma)